In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import os  # Add this import
# Load CSV files
data = pd.read_csv('tpc-ds/1gb/run3/query_execution_times.csv')
data1 = pd.read_csv('tpc-ds/1gb/run3/query1_execution_times.csv')
filename = 'query_metrics_average_case.csv'
query=99

if os.path.exists(filename):
    print(f"Found existing {filename}, removing it...")
    os.remove(filename)
# Clean column names
data.columns = data.columns.str.strip()
data1.columns = data1.columns.str.strip()

# Clean the timestamp strings in data1
data1['start at (ms)'] = data1['start at (ms)'].str.strip()

# Function to convert timestamp string to datetime object
def convert_to_datetime(time_str):
    return datetime.strptime(time_str.strip(), '%H:%M:%S.%f')

In [8]:
def calculate_metrics(query_id, time_col, usage_col):
    # Chuyển đổi cột thời gian sang datetime nếu chưa chuyển
    data[time_col] = pd.to_datetime(data[time_col])
    data1['start at (ms)'] = pd.to_datetime(data1['start at (ms)'])
    
    # Lấy thời gian bắt đầu và duration của truy vấn từ data1
    query_start =data1.loc[data1['query'] == query_id, 'start at (ms)'].values[0]
    query_duration = data1.loc[data1['query'] == query_id, 'time (ms)'].values[0] 
    
    # Tính thời gian kết thúc của truy vấn (chuyển duration từ ms sang timedelta)
    query_end = query_start + pd.to_timedelta(int(query_duration), unit='ms')
    
    # Lọc các dòng trong data có thời gian nằm trong khoảng của truy vấn
    filtered_rows = data[(data[time_col] >= query_start) & (data[time_col] <= query_end)]
    if filtered_rows.empty and query_duration != 0:
        # Get timestamps before query_end
        before_end = data[data[time_col] <= query_end]
        if not before_end.empty:
            # Get the row with the maximum (latest) timestamp
            filtered_rows = before_end[before_end[time_col] == before_end[time_col].max()]
    if not filtered_rows.empty:
        avg_usage = filtered_rows[usage_col].mean()
        return avg_usage
    else:
        return None

In [9]:
def process_query_metrics(query_id):
    # Ví dụ sử dụng:
    avg_cpu_usage = calculate_metrics(query_id, 'CPU_time(ms)', 'CPU_USAGE(%)')
    avg_gpu_usage = calculate_metrics(query_id, 'GPU_time(ms)', 'GPU_USAGE(%)')
    avg_ram_usage = calculate_metrics(query_id, 'RAM_time(ms)', 'RAM_USED(Gb)')
    avg_vram_usage = calculate_metrics(query_id, 'VRAM_time(ms)', 'VRAM_USED(Gb)')
    avg_power_usage = calculate_metrics(query_id, 'POWER_time(ms)', 'POWER_USE(W)')
    # Create a DataFrame with the required data
    query1_duration = data1.loc[data1['query'] == query_id, 'time (ms)'].values[0]
    new_data = pd.DataFrame({
        'query_id': [query_id],
        'time (ms)': [query1_duration],
        'avg_cpu_usage': [avg_cpu_usage],
        'avg_gpu_usage': [avg_gpu_usage],
        'avg_ram_usage': [avg_ram_usage],
        'avg_vram_usage': [avg_vram_usage],
        'avg_power_usage': [avg_power_usage]
    })
    
    try:
        # Try to read existing file
        existing_data = pd.read_csv(filename)
        # Append new data
        output_data = pd.concat([existing_data, new_data], ignore_index=True)
    except FileNotFoundError:
        # If file doesn't exist, use just the new data
        output_data = new_data
    # Save the DataFrame to a CSV file
    output_data.to_csv(filename, index=False)

    return new_data

In [10]:
# Process metrics for queries 1 through 10 using a loop
metrics_results = {}  # Dictionary to store results if needed

for i in range(1, query+1):
    query_id = f"query {i}"
    print(f"Processing {query_id}...")
    
    try:
        # Process the query metrics and store the result
        metrics_results[query_id] = process_query_metrics(query_id)
    except Exception as e:
        print(f"Error processing {query_id}: {e}")

print("All queries processed and appended to query_metrics.csv")

Processing query 1...
Processing query 2...
Processing query 3...
Processing query 4...
Error processing query 4: invalid literal for int() with base 10: ' '
Processing query 5...
Error processing query 5: invalid literal for int() with base 10: ' '
Processing query 6...
Processing query 7...
Processing query 8...
Error processing query 8: invalid literal for int() with base 10: ' '
Processing query 9...
Processing query 10...
Error processing query 10: invalid literal for int() with base 10: ' '
Processing query 11...
Error processing query 11: invalid literal for int() with base 10: ' '
Processing query 12...
Error processing query 12: invalid literal for int() with base 10: ' '
Processing query 13...
Processing query 14...
Error processing query 14: invalid literal for int() with base 10: ' '
Processing query 15...
Error processing query 15: invalid literal for int() with base 10: ' '
Processing query 16...
Error processing query 16: invalid literal for int() with base 10: ' '
Proce